In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scipy.io import loadmat
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from tensorflow.keras import layers, Model


In [2]:
import pandas as pd
from scipy.io import loadmat

# Supondo que você tenha as bibliotecas necessárias importadas e os dados carregados como mencionado

dataTT = loadmat(f"Data04-08_TT_UltVag_Cut.mat")
dataVG = loadmat(f"Data04-08_TF_UltVag_Cut.mat")

dataBaseline = dataTT['Baseline']
dataCincoP = dataTT['CincoP']
dataDez = dataTT['DezP']
dataVinte = dataTT['VinteP']

dadosRigidez = pd.DataFrame()

# Adicionar dataBaseline com coluna y_Baseline
dataBaseline_df = pd.DataFrame(dataBaseline)
baseline_test_df = dataBaseline_df.sample(n=200, random_state=42)
dataBaseline_df = dataBaseline_df.drop(baseline_test_df.index)
dataBaseline_df.reset_index(drop=True, inplace=True)  # Redefinir índices
dadosRigidez = pd.concat([dadosRigidez, dataBaseline_df], ignore_index=True)

# Adicionar dataCincoP com coluna y_CincoP
dataCincoP_df = pd.DataFrame(dataCincoP)
cinco_test_df = dataCincoP_df.sample(n=200, random_state=42)
dataCincoP_df = dataCincoP_df.drop(cinco_test_df.index)
dataCincoP_df.reset_index(drop=True, inplace=True)  # Redefinir índices
dadosRigidez = pd.concat([dadosRigidez, dataCincoP_df], ignore_index=True)

# Adicionar dataDez com coluna y_DezP
dataDez_df = pd.DataFrame(dataDez)
dez_test_df =  dataDez_df.sample(n=200, random_state=42)
dataDez_df = dataDez_df.drop(dez_test_df.index)
dataDez_df.reset_index(drop=True, inplace=True)  # Redefinir índices
dadosRigidez = pd.concat([dadosRigidez, dataDez_df], ignore_index=True)

# Adicionar dataVinte com coluna y_VinteP
dataVinte_df = pd.DataFrame(dataVinte)
vinte_test_df =  dataVinte_df.sample(n=200, random_state=42)
dataVinte_df = dataVinte_df.drop(vinte_test_df.index)
dataVinte_df.reset_index(drop=True, inplace=True)  # Redefinir índices
dadosRigidez = pd.concat([dadosRigidez, dataVinte_df], ignore_index=True)

X_train = dadosRigidez
X_test = pd.concat([baseline_test_df, cinco_test_df, dez_test_df, vinte_test_df], ignore_index=True)

num_terms = X_train.columns.shape[0]

# Reformulando os dados para se adequarem à entrada da rede
X_train = X_train.values.reshape(-1, num_terms, 1)
X_test = X_test.values.reshape(-1, num_terms, 1)


In [7]:
stacked_encoder = Sequential([
            layers.Input(shape=(num_terms,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(16, activation='relu')
            ])

stacked_decoder = Sequential([
            layers.Dense(16, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(64, activation='relu'),    
            layers.Dense(num_terms, activation='linear')
            ])

autoencoder = Sequential([stacked_encoder, stacked_decoder])

autoencoder.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='mse')

autoencoder.fit(
            X_train, X_train,
            epochs=30,
            validation_data=(X_test, X_test),
            verbose = 1
        )


Epoch 1/30
100/100 [==============================] - 1s 7ms/step - loss: 0.0053 - val_loss: 0.0014
Epoch 2/30
100/100 [==============================] - 1s 6ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 3/30
100/100 [==============================] - 1s 6ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 4/30
100/100 [==============================] - 1s 6ms/step - loss: 8.0358e-04 - val_loss: 6.6109e-04
Epoch 5/30
100/100 [==============================] - 1s 6ms/step - loss: 6.2556e-04 - val_loss: 5.5584e-04
Epoch 6/30
100/100 [==============================] - 1s 6ms/step - loss: 4.9237e-04 - val_loss: 5.8206e-04
Epoch 7/30
100/100 [==============================] - 1s 6ms/step - loss: 5.2158e-04 - val_loss: 4.9041e-04
Epoch 8/30
100/100 [==============================] - 1s 6ms/step - loss: 5.5223e-04 - val_loss: 6.1744e-04
Epoch 9/30
100/100 [==============================] - 1s 6ms/step - loss: 7.3448e-04 - val_loss: 8.9153e-04
Epoch 10/30
100/100 [==============================]

In [11]:
# Extract the minimum and maximum values of the latent space. 
latent = pd.DataFrame(stacked_encoder.predict(X_train))
mins = latent.min(axis = 0).values
maxs = latent.max(axis = 0).values
mean = latent.mean(axis = 0).values
stddev = latent.std(axis = 0).values
#print(latent.describe())

100/100 [==============================] - 0s 1ms/step


In [12]:
class sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var))*K.exp(log_var/2)+mean